In [ ]:
import json
import os
import shutil

import matplotlib.patches as patches
import matplotlib.pyplot as plt
import pandas as pd
import yaml
from PIL import Image
from tqdm import tqdm

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("/Users/kimyu/Desktop/best.pt")
model

In [ ]:
results = model.predict(source="/Users/kimyu/Desktop/test2/test2-4", conf=0.4, iou=0.5)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
import os

# 사용자 이동 방향 및 속도 설정
user_direction = np.array([0, -1])  # 사용자 이동 방향 벡터 설정
user_speed = 10  # 사용자의 속도 (예시)

# 최소 Optical Flow 크기 설정
MIN_FLOW_MAGNITUDE = 0.01  # 최소 Optical Flow 크기 (조건 완화)
STOPPED_THRESHOLD = 1.1  # 정차로 간주할 속도 크기 기준

# 좌우 각도 범위 설정 (리스트로 변경하여 순서 지정)
angle_ranges = {
    'left': [
        ('주행', (45, 150)),
        ('반대', (150, 245)),
        ('좌회전', (0, 0)),
        ('우회전', (0, 0)),
        ('정차', (0, 5))
    ],
    'right': [
        ('주행', (0, 250)),
        ('반대', (180, 180)),
        ('좌회전', (0, 0)),
        ('우회전', (0, 0)),
        ('정차', (250, 150))
    ]
}

# 각도 계산 함수
def calculate_angle(vector):
    angle = np.degrees(np.arctan2(vector[1], vector[0]))
    if angle < 0:
        angle += 360
    return angle

# 상태 분류 함수 (상태 체크 순서 지정)
def classify_direction(angle, side, previous_status):
    print(f"Debug: Checking angle {angle:.2f} for side '{side}'")
    for status, (min_angle, max_angle) in angle_ranges[side]:
        print(f"Debug: Checking if {min_angle} <= {angle:.2f} <= {max_angle} for status '{status}'")
        if min_angle <= angle <= max_angle:
            print(f"Debug: Matched angle {angle:.2f} to status '{status}'")
            return status
    print(f"Debug: No match found for angle {angle:.2f}, using previous status '{previous_status}'")
    return previous_status  # 각도가 설정되지 않은 경우 이전 상태 반환

# Optical Flow 계산 함수
def dense_optical_flow(img1, img2, box):
    x1, y1, x2, y2 = map(int, box)
    height, width = img1.shape[:2]
    x1, y1, x2, y2 = max(0, x1), max(0, y1), min(width, x2), min(height, y2)

    img1_cropped = img1[y1:y2, x1:x2]
    img2_cropped = img2[y1:y2, x1:x2]

    if img1_cropped.size == 0 or img2_cropped.size == 0:
        return np.array([0, 0]), 0, (x1, y1, x2, y2)

    img1_gray = cv2.cvtColor(img1_cropped, cv2.COLOR_BGR2GRAY)
    img2_gray = cv2.cvtColor(img2_cropped, cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(img1_gray, img2_gray, None,
                                        0.5, 3, 15, 3, 5, 1.2, 0)
    flow_x = np.mean(flow[..., 0])
    flow_y = np.mean(flow[..., 1])
    flow_magnitude = np.sqrt(flow_x**2 + flow_y**2)
    return np.array([flow_x, flow_y]), flow_magnitude, (x1, y1, x2, y2)

# Optical Flow와 속도 비교를 포함한 박스 정보 처리 함수
def process_boxes_with_debug(filtered_results, frames):
    num_frames = len(frames)
    num_results = len(filtered_results)
    selected_indices = sorted(random.sample(
        range(min(num_frames - 1, num_results)),
        min(num_frames - 1, num_results) // 3))

    previous_status = "정차"  # 초기 상태 설정

    for i in selected_indices:
        frame1 = frames[i].copy()
        frame2 = frames[i + 1].copy()
        height, width = frame1.shape[:2]
        mid_x = width // 2

        print(f"--- Frame {i} to {i+1} ---")

        if i >= len(filtered_results):
            continue

        for box in filtered_results[i]:
            flow_vector, flow_magnitude, (x1, y1, x2, y2) = dense_optical_flow(
                frame1, frame2, box)
            angle = calculate_angle(flow_vector)
            side = 'left' if (x1 + x2) // 2 < mid_x else 'right'  # 좌우 결정

            # STOPPED_THRESHOLD 조건을 제외하고 각도에 따라 상태 분류
            status = classify_direction(angle, side, previous_status)

            # 상태 저장
            previous_status = status

            # 색상 설정
            if status == "주행":
                box_color = (255, 255, 0)  # 노랑
            elif status == "좌회전":
                box_color = (255, 0, 255)  # 보라
            elif status == "우회전":
                box_color = (255, 192, 203)  # 핑크
            elif status == "반대":
                box_color = (0, 0, 255)  # 빨강
            else:
                box_color = (0, 255, 0)  # 초록 (정차)

            side_text = "왼쪽" if side == "left" else "오른쪽"
            print(f"{side_text} - Flow Vector: [{flow_vector[0]:.4f}, {flow_vector[1]:.4f}], "
                  f"Flow Magnitude: {flow_magnitude:.4f}, Angle: {angle:.2f} degrees, "
                  f"Status: {status}, Box Color: {box_color}")

            # 바운딩 박스와 방향 벡터 표시
            cv2.rectangle(frame1, (x1, y1), (x2, y2), box_color, 2)
            center_x = (x1 + x2) // 2
            center_y = (y1 + y2) // 2
            scale = 10
            end_x = int(center_x + flow_vector[0] * scale)
            end_y = int(center_y + flow_vector[1] * scale)
            cv2.arrowedLine(frame1, (center_x, center_y), (end_x, end_y),
                            (0, 0, 255), 3, cv2.LINE_AA)

        plt.imshow(cv2.cvtColor(frame1, cv2.COLOR_BGR2RGB))
        plt.title(f"Image Index: {i}")
        plt.show()

# 모든 바운딩 박스를 가져오는 함수 (0번 클래스만 필터링)
def get_all_boxes(results):
    filtered_results = []
    for result in results:
        if result.boxes is not None:
            boxes = result.boxes.xyxy.cpu().numpy()
            classes = result.boxes.cls.cpu().numpy()
            zero_class_boxes = [boxes[i] for i in range(len(classes)) if int(classes[i]) == 0]
            filtered_results.append(zero_class_boxes)
        else:
            filtered_results.append([])
    return filtered_results

# 디렉토리에서 프레임 불러오기
def load_frames_from_dir(directory):
    frames = []
    for filename in sorted(os.listdir(directory)):
        if filename.endswith(".png") or filename.endswith(".jpg"):
            img_path = os.path.join(directory, filename)
            frame = cv2.imread(img_path)
            if frame is not None:
                frames.append(frame)
            else:
                print(f"Warning: Could not load image {img_path}")
    return frames

# YOLO 결과를 가져와서 모든 바운딩 박스 필터링
filtered_results = get_all_boxes(results)

# YOLO 결과와 Optical Flow 처리
frames = load_frames_from_dir("/Users/kimyu/Desktop/test2/test2-4")
process_boxes_with_debug(filtered_results, frames)
